In [1]:
import os
import sys
import time
import pickle
import string
from datetime import datetime
from datetime import timedelta
from itertools import chain, cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib
import sqlite3

import numpy as np
import pandas as pd
import dask.dataframe as dd
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import lxml
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
#from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

##https://python-visualization.github.io/folium/docs-v0.6.0/
#https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf
import folium
import folium.plugins
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster


#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display
from IPython.display import clear_output

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python
#https://hackersandslackers.com/dealing-with-xml-in-python/

In [2]:
pwd

'/home/rsouza/Documents/repos/exploreAT-TEI_XML_NLP'

### PART II: Extracting information from XML data files

#### Reading a single XML Data file to test:

In [4]:
#xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
xmlfile =  './TEI-XML-2018/bel_e-tei-02/e206_qdb-TEI-02.xml'

namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [5]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [6]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x7f44d80c3598>


In [7]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [8]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [9]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}figDesc',
 '{http://www.tei-c.org/ns/1.0}figure',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',


In [10]:
#for elem in root.iter():
for elem in root.iter('{http://www.tei-c.org/ns/1.0}entry'):
    for attrName, attrValue in elem.items():
        print(attrName + ' ==> ' + attrValue)
    break
    print(elem.tag)
    print(elem.text)
    print(elem.attrib)
    print(elem.items())
    print('\n')

{http://www.w3.org/XML/1998/namespace}id ==> e206_qdb-d1e2
{http://www.w3.org/XML/1998/namespace}lang ==> bar


In [11]:
root.tag

'{http://www.tei-c.org/ns/1.0}TEI'

#### Iterating through all XML Data files

In [12]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

['bel_s-tei-02',
 'bel_g-tei-02',
 'bel_q-r-tei-02',
 'bel_k-tei-02',
 'bel_u-w-tei-02',
 'bel_e-tei-02',
 'bel_n-o-p-tei-02',
 'bel_l-tei-02',
 'bel_h-tei-02',
 'bel_z-tei-02',
 'bel_i-j-tei-02',
 'bel_f-tei-02',
 'bel_m-tei-02',
 'bel_d-tei-02']

In [21]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('{}{}/{}'.format(datapath, folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles[0:20]):
    print(i+1, '--' ,w)

Files in the folder:
1 -- ./TEI-XML-2018/bel_s-tei-02/s701_qdb-TEI-02.xml
2 -- ./TEI-XML-2018/bel_s-tei-02/s789_qdb-TEI-02.xml
3 -- ./TEI-XML-2018/bel_s-tei-02/s712_qdb-TEI-02.xml
4 -- ./TEI-XML-2018/bel_s-tei-02/s757_qdb-TEI-02.xml
5 -- ./TEI-XML-2018/bel_s-tei-02/s811_qdb-TEI-02.xml
6 -- ./TEI-XML-2018/bel_s-tei-02/s743_qdb-TEI-02.xml
7 -- ./TEI-XML-2018/bel_s-tei-02/s697_qdb-TEI-02.xml
8 -- ./TEI-XML-2018/bel_s-tei-02/s751_qdb-TEI-02.xml
9 -- ./TEI-XML-2018/bel_s-tei-02/s729_qdb-TEI-02.xml
10 -- ./TEI-XML-2018/bel_s-tei-02/s685_qdb-TEI-02.xml
11 -- ./TEI-XML-2018/bel_s-tei-02/s761_qdb-TEI-02.xml
12 -- ./TEI-XML-2018/bel_s-tei-02/s719_qdb-TEI-02.xml
13 -- ./TEI-XML-2018/bel_s-tei-02/s698_qdb-TEI-02.xml
14 -- ./TEI-XML-2018/bel_s-tei-02/s688_qdb-TEI-02.xml
15 -- ./TEI-XML-2018/bel_s-tei-02/s776_qdb-TEI-02.xml
16 -- ./TEI-XML-2018/bel_s-tei-02/s708_qdb-TEI-02.xml
17 -- ./TEI-XML-2018/bel_s-tei-02/s740_qdb-TEI-02.xml
18 -- ./TEI-XML-2018/bel_s-tei-02/s807_qdb-TEI-02.xml
19 -- ./TEI-XML-

In [23]:
t0 = time.time()

columns=['filename',
         'entry_id',
         'date',
         'usg_corresp', 
         'simple_placeName',
         'orig_placeName',
         'placeName_id',
         'Bundesland',
         'Bundesland_idno',
         'Großregion',
         'Großregion_idno',
         'Kleinregion',
         'Kleinregion_idno',
         'Gemeinde',
         'Gemeinde_idno',
         'Ort',
         'Ort_idno',
         'hauptlemma_unique',
         'hauptlemma_orig',
         'hauptlemma_normalized',
         'nebenlemma_unique',
         'nebenlemma_orig',
         'nebenlemma_normalized',
         'verweise',
         'archiv',
         'quelle',
         'quelleBearbeitet',
         'seite',
         'fragebogenNummer',
         'paragraph',
         'bibl',
         'tustep',
         'pos',
         'etym',
         'note_notabene',
         'note_anmerkung',
         'note_diverse',
         'citation'
         'cit_type',
         'cit_quote',
         'cit_def',
         'cit_interp',
         'cit_ref',
         'cit_pRef',
         'cit_note',
         'cit_re',
         'cit_certainty',
         'sense',
         'ref',
         'ref_pRef',
         'ref_date',
         'ref_num',
         'ref_certainty',
         'figure'
        ]

df_lemmas = pd.DataFrame()
list_records = []

for fileid, teixmlfile in enumerate(onlyfiles):
    #print(teixmlfile)
    root = ET.parse(teixmlfile).getroot()
    for entry in root.iter('{http://www.tei-c.org/ns/1.0}entry'):

        record = {'filename': teixmlfile,
                  'entry_id': None,
                  'date': None,
                  'usg_corresp': None,
                  'simple_placeName': None,
                  'orig_placeName': None,
                  'placeName_id': None,
                  'Bundesland': None,
                  'Bundesland_idno': None,
                  'Großregion': None,
                  'Großregion_idno': None,
                  'Kleinregion': None,
                  'Kleinregion_idno': None,
                  'Gemeinde': None,
                  'Gemeinde_idno': None,
                  'Ort': None,
                  'Ort_idno': None,
                  'hauptlemma_unique': None,
                  'hauptlemma_orig': None,
                  'hauptlemma_normalized': None,
                  'nebenlemma_unique': None,
                  'nebenlemma_orig': None,
                  'nebenlemma_normalized': None,
                  'verweise': None,
                  'archiv': None,
                  'quelle': None,
                  'quelleBearbeitet': None,
                  'seite': None,
                  'fragebogenNummer': None,
                  'paragraph': None,
                  'bibl': [],
                  'tustep': [],
                  'pos': None,
                  'etym': [],
                  'note_notabene': None,
                  'note_anmerkung': None,
                  'note_diverse': None,
                  'citation': [],
                  'cit_quote': [],
                  'cit_def': [],
                  'cit_interp': [],
                  'cit_ref': [],
                  'cit_pRef': [],
                  'cit_note': [],
                  'cit_re':[],
                  'cit_certainty':[],
                  'sense': [],
                  'ref': [],
                  'ref_pRef': [],
                  'ref_date': [],
                  'ref_num': [],
                  'ref_certainty': [],
                  'figure': None,
                 }

        record['entry_id'] = entry.attrib['{http://www.w3.org/XML/1998/namespace}id']
        
        for child in entry.iter():
            level = 'simple_placeName'
            if child.tag == '{http://www.tei-c.org/ns/1.0}usg':
                if 'corresp' in child.attrib:
                    record['usg_corresp'] = child.attrib['corresp']
                for subchild in child.iter():
                    if 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}placeName'\
                            and subchild.attrib['type'] == 'orig':
                        record['orig_placeName'] = subchild.text

                    elif 'ref' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}listPlace':
                        record['placeName_id'] = subchild.attrib['ref']

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Bundesland':
                        level = 'Bundesland'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Großregion':
                        level = 'Großregion'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Kleinregion':
                        level = 'Kleinregion'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Gemeinde':
                        level = 'Gemeinde'

                    elif 'type' in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}place' \
                            and subchild.attrib['type'] == 'Ort':
                        level = 'Ort'

                    if 'type' not in subchild.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}placeName':
                        record[level] = subchild.text
                    elif 'type' in child.attrib \
                            and subchild.tag == '{http://www.tei-c.org/ns/1.0}idno':
                        record['{}_idno'.format(level)] = subchild.text

            elif child.tag == '{http://www.tei-c.org/ns/1.0}form' \
                        and 'type' in child.attrib:
                if child.attrib['type'] == 'hauptlemma':
                    for subchild in child.iter():
                        if subchild.tag == '{http://www.tei-c.org/ns/1.0}orth':
                            if 'type' not in subchild.attrib:
                                record['hauptlemma_unique'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'orig':
                                record['hauptlemma_orig'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'normalized':
                                record['hauptlemma_normalized'] = subchild.text
                elif child.attrib['type'] == 'nebenlemma':
                    for subchild in child.iter():
                        if subchild.tag == '{http://www.tei-c.org/ns/1.0}orth':
                            if 'type' not in subchild.attrib:
                                record['nebenlemma_unique'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'orig':
                                record['nebenlemma_orig'] = subchild.text
                            elif 'type' in subchild.attrib and subchild.attrib['type'] == 'normalized':
                                record['nebenlemma_normalized'] = subchild.text       
                elif child.attrib['type'] == 'lautung':
                    for subchild in child.iter():
                            if 'notation' in subchild.attrib and subchild.attrib['notation'] == 'tustep':
                                record['tustep'].append(subchild.text)

            elif child.tag == '{http://www.tei-c.org/ns/1.0}pos':
                record['pos'] = child.text
            
            elif child.tag == '{http://www.tei-c.org/ns/1.0}date':
                record['date'] = child.text
                
            elif child.tag == '{http://www.tei-c.org/ns/1.0}note':
                if 'type' in child.attrib and child.attrib['type'] == 'notabene':
                    record['note_notabene'] = subchild.text
                elif 'type' in child.attrib and child.attrib['type'] == 'anmerkung':
                    record['note_anmerkung'] = child.text
                elif 'type' in child.attrib and child.attrib['type'] == 'diverse':
                    record['note_diverse'] = child.text
                elif 'type' in child.attrib:
                    print('new note type ', child.attrib['type'] )

            elif child.tag == '{http://www.tei-c.org/ns/1.0}cit':
                for subchild in child.iter():
                    if subchild.tag == '{http://www.tei-c.org/ns/1.0}cit':
                        record['citation'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}quote':
                        record['cit_quote'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}def':
                        record['cit_def'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}interp':
                        record['cit_interp'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}ref':
                        record['cit_ref'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}pRef':
                        record['cit_pRef'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}note':
                        record['cit_note'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}re':
                        record['cit_re'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}certainty':
                        record['cit_certainty'].append(subchild.text)
                    else:
                        print('new cit tag ', subchild.tag)                
                    
            elif child.tag == '{http://www.tei-c.org/ns/1.0}ref':
                for subchild in child.iter():
                    if subchild.tag == '{http://www.tei-c.org/ns/1.0}ref':
                        record['ref'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}bibl':
                        record['bibl'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}certainty':
                        record['ref_certainty'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}pRef':
                        record['ref_pRef'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}date':
                        record['ref_date'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}num':
                        record['ref_num'].append(subchild.text)
                    elif subchild.tag == '{http://www.tei-c.org/ns/1.0}certainty':
                        record['ref_certainty'].append(subchild.text)
                    else:
                        print('new ref tag ', subchild.tag)  
                    #print(subchild.tag, subchild.text)
                if child.attrib['type'] == 'verweise':
                    record['verweise'] = child.text
                elif child.attrib['type'] == 'archiv':
                    record['archiv'] = child.text
                elif child.attrib['type'] == 'quelle':
                    record['quelle'] = child.text
                elif child.attrib['type'] == 'quelleBearbeitet':
                    record['quelleBearbeitet'] = child.text
                elif child.attrib['type'] == 'seite':
                    record['seite'] = child.text
                elif child.attrib['type'] == 'fragebogenNummer':
                    record['fragebogenNummer'] = child.text
                elif child.attrib['type'] == 'paragraph':
                    record['paragraph'] = child.text
                elif child.attrib['type'] == 'bibl':
                    record['bibl'].append(subchild.text)
                else:
                    print('new ref type --> ', child.attrib['type'])
            
            elif child.tag == '{http://www.tei-c.org/ns/1.0}etym':
                for subchild in child.iter():
                    record['etym'].append(subchild.text)
                    
            elif child.tag == '{http://www.tei-c.org/ns/1.0}sense':
                for subchild in child.iter():
                    record['sense'].append(subchild.text)
                    
            elif child.tag == '{http://www.tei-c.org/ns/1.0}figure':
                for subchild in child.iter():
                    if subchild.tag == '{http://www.tei-c.org/ns/1.0}figDesc':
                        record['figure']= subchild.text
                        print(subchild.text)
                    
        list_records.append(record)
        
    if fileid%20 == 0:
        clear_output()
        print('Processing time is {0:.2f} minutes for file {1}'.format(((time.time() - t0) / 60), fileid))
        
with open('./list_lemmas.pkl', 'wb') as f:
    pickle.dump(list_records, f)
print('Total Processing time was {0:.2f} minutes'.format(((time.time() - t0) / 60)))

Processing time is 6.47 minutes for file 720
./TEI-XML-2018/bel_d-tei-02/d158_qdb-TEI-02.xml
./TEI-XML-2018/bel_d-tei-02/d171_qdb-TEI-02.xml
Skizze siehe HKt171 01.tif, HK
entlang der Längsseiten des HausesSkizze siehe HKt171 02.tif, HK
Stellen FÜR Milchschüsseln MIT drehbarem Baum vom Boden bis zur DeckeSkizze siehe HKt171 03.tif, HK
Skizze siehe HKt171 04.tif, HK
s. (Holzknecht)seldet171 05.tif, HK (ohne Zeichnung)
./TEI-XML-2018/bel_d-tei-02/d159_qdb-TEI-02.xml
d159 01.tif, HK
d159 02.tif, HK
d159 03.tif, HK
Zeichng. HKd159 04.tif, HK
Zeichng. HKd159 05.tif, HK
Zeichng. HKd159 06.tif, HK
./TEI-XML-2018/bel_d-tei-02/d183_qdb-TEI-02.xml
Zeichng. HKt183 01.tif, HK
Aus einem Stück zarten Holzes gearbeitet; aus weichem Holz.t183 02.tif, HK
Zeichng. HKt183 03.tif, HK
Manche Backtröge sind wie große Waschtröge; sie stehen auf 2 Stühlen.Zeichng. HKt183 04.tif, HK
Zeichng. HKt183 05.tif, HK
Zeichng. HKt183 06.tif, HK
Der Backtrog bei gewerbl. Bäckern hat obige Form, ist 2-3 m lang, 0.5 m tie

In [24]:
len(list_records)

2416499

In [25]:
df_lemmas = pd.DataFrame(list_records)

In [26]:
df_lemmas.info(verbose=True, null_counts=True, memory_usage='deep') #, max_cols=100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2416499 entries, 0 to 2416498
Data columns (total 53 columns):
filename                 2416499 non-null object
entry_id                 2416499 non-null object
date                     65060 non-null object
usg_corresp              16675 non-null object
simple_placeName         395816 non-null object
orig_placeName           1334384 non-null object
placeName_id             1316887 non-null object
Bundesland               1316889 non-null object
Bundesland_idno          1316889 non-null object
Großregion               1296722 non-null object
Großregion_idno          1225463 non-null object
Kleinregion              1286463 non-null object
Kleinregion_idno         1209145 non-null object
Gemeinde                 1198447 non-null object
Gemeinde_idno            0 non-null object
Ort                      1198447 non-null object
Ort_idno                 1198447 non-null object
hauptlemma_unique        985853 non-null object
hauptlemma_orig  

In [27]:
del df_lemmas['Gemeinde_idno']
for entry in df_lemmas.columns:
    df_lemmas[entry] = df_lemmas[entry].apply(lambda x:x if x != [] else None)

df_lemmas['tustep'] = df_lemmas['tustep'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['etym'] = df_lemmas['etym'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_def'] = df_lemmas['cit_def'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_interp'] = df_lemmas['cit_interp'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_note'] = df_lemmas['cit_note'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_pRef'] = df_lemmas['cit_pRef'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_quote'] = df_lemmas['cit_quote'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_re'] = df_lemmas['cit_re'].apply(lambda x:x if x == None else tuple(x))
df_lemmas['cit_ref'] = df_lemmas['cit_ref'].apply(lambda x:x if x == None else tuple(x))
#df_lemmas['citation'] = df_lemmas['citation'].apply(lambda x:x if x == None else tuple(x))
#df_lemmas['cit_certainty'] = df_lemmas['cit_certainty'].apply(lambda x:x if x == None else tuple(x))
#df_lemmas['ref_certainty'] = df_lemmas['ref_certainty'].apply(lambda x:x if x == None else tuple(x))

del df_lemmas['citation']
del df_lemmas['cit_certainty']
del df_lemmas['ref_certainty']

for entry in df_lemmas.columns:
    df_lemmas[entry] = df_lemmas[entry].apply(lambda x:x if x != () else None)

In [28]:
df_lemmas.columns

Index(['filename', 'entry_id', 'date', 'usg_corresp', 'simple_placeName',
       'orig_placeName', 'placeName_id', 'Bundesland', 'Bundesland_idno',
       'Großregion', 'Großregion_idno', 'Kleinregion', 'Kleinregion_idno',
       'Gemeinde', 'Ort', 'Ort_idno', 'hauptlemma_unique', 'hauptlemma_orig',
       'hauptlemma_normalized', 'nebenlemma_unique', 'nebenlemma_orig',
       'nebenlemma_normalized', 'verweise', 'archiv', 'quelle',
       'quelleBearbeitet', 'seite', 'fragebogenNummer', 'paragraph', 'bibl',
       'tustep', 'pos', 'etym', 'note_notabene', 'note_anmerkung',
       'note_diverse', 'cit_quote', 'cit_def', 'cit_interp', 'cit_ref',
       'cit_pRef', 'cit_note', 'cit_re', 'sense', 'ref', 'ref_pRef',
       'ref_date', 'ref_num', 'figure'],
      dtype='object')

In [29]:
df_lemmas.to_pickle('./df_lemmas.pkl')

#### Building a SQLite Database

- From Dataframe

In [ ]:
df_lemmas = pd.read_pickle('./df_lemmas.pkl')

In [ ]:
conn = sqlite3.connect('./lemmas_db1.sqlite')

In [ ]:
df_lemmas.applymap(str).head()

In [ ]:
df_lemmas.applymap(str).to_sql(name='lemmas', con=conn)

In [ ]:
conn.close()

- From list of dicts

In [ ]:
with open('./list_lemmas.pkl', 'rb') as f:
    list_lemmas = pickle.load(f)

In [ ]:
list_lemmas[0]

In [ ]:
conn = sqlite3.connect('./lemmas_db.sqlite')
cur = conn.cursor()

In [ ]:
q = '''
CREATE TABLE IF NOT EXISTS lemmas (
Gemeinde_idno text,
hauptlemma_orig text,
quelle text,
bibl text,
pos text,
Kleinregion_idno text,
cit_type text,
etym text,
cit_quote text,
verweise text,
Bundesland_idno text,
tustep text,
note_diverse text,
orig_placeName text,
note_anmerkung text,
nebenlemma_unique text,
Großregion text,
quelleBearbeitet text,
archiv text,
Kleinregion text,
placeName_id text,
seite text,
Gemeinde text,
Ort_idno text,
cit_note text,
Ort text,
cit_re text,
hauptlemma_unique text,
cit_interp text,
cit_def text,
Bundesland text,
date text,
nebenlemma_normalized text,
hauptlemma_normalized text,
paragraph text,
entry_id text,
nebenlemma_orig text,
certainty text,
Großregion_idno text,
cit_ref text,
fragebogenNummer text,
note_notabene text,
usg_corresp text,
filename text,
simple_placeName text,
cit_pRef text
)
'''

In [ ]:
# Create table
cur.execute(q)

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

In [ ]:
cur.execute('select count(*) from lemmas')
print(cur.fetchall())

In [ ]:
def post_row(conn, tablename, rec):
    keys = ','.join(rec.keys())
    question_marks = ','.join(list('?'*len(rec)))
    values = [str(v) for v in rec.values()]
    values = tuple(values)
    conn.execute('INSERT INTO '+tablename+' ('+keys+') VALUES ('+question_marks+')', values)

In [ ]:
for idno, entry in enumerate(list_lemmas):
    post_row(conn, 'lemmas', entry)
    if idno%100000 == 0:
        print(idno)
conn.commit()
#conn.close()